In [3]:
# 프로젝트 디렉토리에서 실행
# jupyter notebook 에서 django sell을 사용할 수 있도록 설정
import os
os.environ['DJANGO_SETTINGS_MODULE'] = 'config.settings'# settings파일이 어디있는가 알려주어야함.
os.environ['DJANGO_ALLOW_ASYNC_UNSAFE'] = 'true'

import django
django.setup()


In [4]:
from polls.models import Question, Choice

## 전체조회
- all()

In [5]:
#MODEL클래스이름.objects.xxxx

qs = Question.objects.all()


In [6]:
#QuerySet.query : sql문 조회
print(qs.query)

SELECT "polls_question"."id", "polls_question"."question_text", "polls_question"."pub_date" FROM "polls_question"


In [7]:
for q in qs:
    print(q,type(q))

좋아하는 색깔은 무엇입니까? <class 'polls.models.Question'>
가고 싶은 나라를 선택하세요. <class 'polls.models.Question'>
좋아하는 동물은 무엇입니까? <class 'polls.models.Question'>


In [8]:
# 특정 조회결과 하나 조회 - indexing
# 범위 조회 : slicing
qs[0]

<Question: 좋아하는 색깔은 무엇입니까?>

In [9]:
qs[1:]

[<Question: 가고 싶은 나라를 선택하세요.>, <Question: 좋아하는 동물은 무엇입니까?>]

In [10]:
qs[-1] # 음수 인덱스는 사용 못함

AssertionError: Negative indexing is not supported.

In [11]:
qs.first()


<Question: 좋아하는 색깔은 무엇입니까?>

In [12]:
qs.last()

<Question: 좋아하는 동물은 무엇입니까?>

In [13]:
qs=Choice.objects.all()
qs.order_by('choice_text')


<QuerySet [<Choice: 강아지>, <Choice: 검정색>, <Choice: 미국>, <Choice: 영국>, <Choice: 중국>, <Choice: 파랑색>, <Choice: 펭귄>]>

In [14]:
qs = Choice.objects.all().order_by('choice_text')
qs

<QuerySet [<Choice: 강아지>, <Choice: 검정색>, <Choice: 미국>, <Choice: 영국>, <Choice: 중국>, <Choice: 파랑색>, <Choice: 펭귄>]>

In [15]:
c = qs.first()
print(type(c))

<class 'polls.models.Choice'>


In [16]:
# 조회한 model의 속성을 조회 => 특정 컬럼값 조회 => . 표기법
#Choice : id, choice.test, vote, question_id
#"polls_choice"."id", "polls_choice"."choice_text", "polls_choice"."vote", "polls_choice"."question_id" FROM "polls_choice" ORDER BY "polls_choice"."choice_text" 

for q in qs:
    print(q.id, q.choice_text, q.vote, q.question_id, sep='\t')

6	강아지	0	3
1	검정색	2	1
4	미국	1	2
3	영국	0	2
5	중국	0	2
2	파랑색	0	1
7	펭귄	1	3


# where 절
- filter(조회조건)
    - 조회조건을 만족하는 0개 이상의 행을 조회
- exclude(조회조건)
    - filter의 반대. 조회조건을 만족하지 않는 0개 이상의 행을 조회 (not)
- get(조회조건)
    - 조회조건을 만족하는 1개 행을 조회 (없거나, 2개 이상이면 오류)
- filter/exclude : QuerySet으로 반환
- get : Model 객체를 반환

In [17]:
#where id =1
#qs = Question.objects.filter(id=1)
#pk값은 컬럼 조회 : 컬럼명, pk
qs = Question.objects.filter(pk=3)
qs

#결과가 Queryset이다. 여기서 다른 것을 조회할 순 없다

<QuerySet [<Question: 좋아하는 동물은 무엇입니까?>]>

In [18]:
print(qs.first().pub_date)

2021-05-26 17:23:56.004295+00:00


In [19]:
quest = Question.objects.get(pk=2)
print(type(quest))

<class 'polls.models.Question'>


In [20]:
try:
    r=Question.objects.get(id=1)
    print(r.pub.date)
except:
    print('조회결과가 없습니다.')
    
#except 가 될 수 있으니 그냥 하지마라.

조회결과가 없습니다.


In [21]:
Question.objects.get(id__gt=1) # 조회결과가 2개 이상일 경우 예외 발생

MultipleObjectsReturned: get() returned more than one Question -- it returned 2!

In [22]:
# 조회조건
#qs=Choice.objects.filter(id__gt=3) # id>3
#qs=Choice.objects.filter(id__gte=5) #id>=5
qs=Choice.objects.filter(id__lte=5) # id<=5
for q in qs:
    print(q.id,q.choice_text)

1 검정색
2 파랑색
3 영국
4 미국
5 중국


In [23]:
#qs = Question.objects.filter(question_text__startswith='가고') # q_t like '가고%'
#qs = Question.objects.filter(question_text__endswith='입니까?') # q_t like '%입니까'
qs = Question.objects.filter(question_text__contains='무엇') # q_t like '%무엇%'
for q in qs:
    print(q.id,q)

1 좋아하는 색깔은 무엇입니까?
3 좋아하는 동물은 무엇입니까?


In [24]:
#qs = Choice.objects.filter(id__in = [2,6,7,20]) # where id in [2,6,7,20]
qs = Choice.objects.filter(id__range=[3,6]) # between 3 and 6
print(qs.query)
for q in qs:
    print(q.id,q)

SELECT "polls_choice"."id", "polls_choice"."choice_text", "polls_choice"."vote", "polls_choice"."question_id" FROM "polls_choice" WHERE "polls_choice"."id" BETWEEN 3 AND 6
3 영국
4 미국
5 중국
6 강아지


In [25]:
qs= Choice.objects.filter(pk=3, choice_text="영국")

qs.first().id, qs.first().choice_text

(3, '영국')

In [26]:
qs = Choice.objects.filter(pk__lt=5, choice_text__contains='색')
for q in qs:
    print(q.id, q)

1 검정색
2 파랑색


In [27]:
#OR 조건 Q()
from django.db.models import Q
#qs = Choice.objects.filter(Q(pk__gt=5)|Q( choice_text__contains='색'))
qs = Choice.objects.filter(Q(pk__gt=5)|~Q( choice_text__contains='색'))
#qs = Choice.objects.filter(~Q(pk=5))
print(qs.query)

for q in qs:
    print(q.id, q)

SELECT "polls_choice"."id", "polls_choice"."choice_text", "polls_choice"."vote", "polls_choice"."question_id" FROM "polls_choice" WHERE ("polls_choice"."id" > 5 OR NOT ("polls_choice"."choice_text" LIKE %색% ESCAPE '\'))
3 영국
4 미국
5 중국
6 강아지
7 펭귄


In [28]:
#qs =Choice.objects.exclude(pk=5)
qs =Choice.objects.exclude(pk__lt=5)
print(qs.query)#5보다 작은 것 제외
for q in qs:
    print(q.id, q)

SELECT "polls_choice"."id", "polls_choice"."choice_text", "polls_choice"."vote", "polls_choice"."question_id" FROM "polls_choice" WHERE NOT ("polls_choice"."id" < 5)
5 중국
6 강아지
7 펭귄


# 컬럼 선택
select 컬럼,컬럼
- values(field이름,...)
     - 개별 조회결과를 Dictionary에 넣어서 반환
     

In [29]:
qs = Choice.objects.all().values('choice_text','vote')
print(qs.query)
for q in qs:
    print(q['choice_text'],q['vote'])


SELECT "polls_choice"."choice_text", "polls_choice"."vote" FROM "polls_choice"
검정색 2
파랑색 0
영국 0
미국 1
중국 0
강아지 0
펭귄 1


In [31]:
qs = Choice.objects.all() # 이때는 쿼리가 실행된 것이 아님.
qs2 = qs.values('choice_text') # 그냥 쿼리를 만들고 있는 중
for q in qs: # 이때 실행하는 것
    print(q)
    

검정색
파랑색
영국
미국
중국
강아지
펭귄


# 집계
- 단순 집계
    - aggregate(집계함수(), ....)
- group by
    - values(그룹으로 묶을 컬럼).annotate(집계함수(),....)
- 결과를 dictionary로 반환

In [32]:
#조회결과 행수 조회
qs = Choice.objects.all()
qs= Choice.objects.filter(id__lt=3)
len(qs), qs.count()

(2, 2)

In [33]:
from django.db.models import Count, Sum, Avg, Min, Max, StdDev, Variance

In [38]:
#dic = Choice.objects.aggregate(Count("id")) # select count('id') from choice
dic = Choice.objects.aggregate(Count("id"), Count('choice_text'), Sum('vote'), Avg('vote'),Min('vote'),Max('vote'), StdDev('vote'),Variance('vote') )
print(type(dic))
dic

<class 'dict'>


{'id__count': 10,
 'choice_text__count': 10,
 'vote__sum': 78,
 'vote__avg': 7.8,
 'vote__min': 2,
 'vote__max': 20,
 'vote__stddev': 5.491812087098392,
 'vote__variance': 30.159999999999997}

In [39]:
dic = Question.objects.aggregate(Min('pub_date'), Max('pub_date'))
dic

{'pub_date__min': datetime.datetime(2021, 5, 26, 17, 19, 14, 753123, tzinfo=<UTC>),
 'pub_date__max': datetime.datetime(2021, 5, 26, 17, 23, 56, 4295, tzinfo=<UTC>)}

In [41]:
print(dic['pub_date__min'])
dic['pub_date__min'].strftime('%Y년 %m월 %d일')

2021-05-26 17:19:14.753123+00:00


'2021년 05월 26일'

In [45]:
#where +집계 -> filter를 먼저 한 후, .집계
dic = Choice.objects.filter(id__gt=5).aggregate(Sum('vote'))
dic

{'vote__sum': 54}

In [ ]:
#1)from 2)where 3) group by 4) having by 5)order by

In [51]:
# group by
#choice => 질문별 vote의 합계, 평균
#그룹별 집계 : dictionary, 최종반환 : QuerySet
qs = Choice.objects.values('question_id').annotate(Sum('vote'),Avg('vote'))
qs

<QuerySet [{'question_id': 1, 'vote__sum': 18, 'vote__avg': 6.0}, {'question_id': 2, 'vote__sum': 50, 'vote__avg': 10.0}, {'question_id': 3, 'vote__sum': 10, 'vote__avg': 5.0}]>

In [54]:
#d[변수]꼴로
for d in qs:
    print(f"질문 : {d['question_id']}, 투표합계 : {d['vote__sum']}, 투표평균 : {d['vote__avg']} ")

질문 : 1, 투표합계 : 18, 투표평균 : 6.0 
질문 : 2, 투표합계 : 50, 투표평균 : 10.0 
질문 : 3, 투표합계 : 10, 투표평균 : 5.0 


# 테이블간의 관계

## 자식테이블에서 부모 테이블 값을 조회

In [60]:
c1 = Choice.objects.get(pk=3)
type(c1) # c1 : choice모델 객체

polls.models.Choice

In [66]:
# 모델에서 속성들 조회 -> 각 컬럼들 값 조회
c1.id, c1.pk, c1.choice_text, c1.vote, c1.question_id

(3, 3, '영국', 5, 2)

In [69]:
q1=c1.question # 부모 모델 객체 반환
type(q1),q1

(polls.models.Question, <Question: 가고 싶은 나라를 선택하세요.>)

In [70]:
print(q1.question_text, q1.pub_date)

가고 싶은 나라를 선택하세요. 2021-05-26 17:19:46.670356+00:00


## 부모테이블에서 자식 테이블 값을 조회

In [73]:
q = Question.objects.get(pk=3)

type(q),q

(polls.models.Question, 3)

In [74]:
q.id, q.question_text, q.pub_date

(3,
 '좋아하는 동물은 무엇입니까?',
 datetime.datetime(2021, 5, 26, 17, 23, 56, 4295, tzinfo=<UTC>))

In [80]:
c=q.choice_set
# Choice.objects와 같으나, q와 관계된 것만 조회할 수 있음
type(c)

django.db.models.fields.related_descriptors.create_reverse_many_to_one_manager.<locals>.RelatedManager

In [81]:
c.all()

<QuerySet [<Choice: 강아지>, <Choice: 펭귄>]>

In [82]:
c.filter(choice_text='강아지')

<QuerySet [<Choice: 강아지>]>

# insert/update/delete
- insert/update
    - 모델 객체 (insert할 내용을 가진).save()
- delete
    - 모델 객체(삭제할 pk를 가진).delete()

In [102]:
#insert
qs=Question.objects.all()
for q in qs:
    print(q.pk)

1
2
3
4


In [85]:
from datetime import datetime, timedelta

In [116]:
# pk : id - 자동증가(생략)
#id 값은 생략하는 것이 좋다.
new_q = Question()
new_q.question_text = "좋아하는 색은 무엇입니까?"
new_q.pub_date=datetime.now()
new_q.question_text,new_q.pub_date,new_q.pk

('좋아하는 색은 무엇입니까?', datetime.datetime(2021, 5, 27, 14, 39, 5, 34257), None)

In [117]:
new_q.save()

In [119]:
q = Question.objects.get(pk = 8)
q.question_text= "좋아하는 색은 무엇입니까"
q.pub_date=datetime.now() - timedelta(days=100)
print(q.question_text, q.pub_date)
q.save()

좋아하는 색은 무엇입니까 2021-02-16 14:40:05.773827


c:\users\rey\anaconda3_32bit\envs\django\lib\site-packages\django\db\models\fields\__init__.py:1416: RuntimeWarning: DateTimeField Question.pub_date received a naive datetime (2021-02-16 14:40:05.773827) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"


In [120]:
qs = Question.objects.all()
for q in qs:
    print(q.pk, q.question_text, q.pub_date)

2 가고 싶은 나라를 선택하세요. 2021-05-26 17:19:46.670356+00:00
3 좋아하는 동물은 무엇입니까? 2021-05-26 17:23:56.004295+00:00
4  좋아하는 가수는 누구입니까 2021-02-16 05:31:14.059220+00:00
8 좋아하는 색은 무엇입니까 2021-02-16 05:40:05.773827+00:00


In [107]:
# 삭제

d_q = Question()
d_q.id = 1
d_q.id, d_q.question_text, d_q.pub_date

(1, '', None)

In [108]:
d_q.delete()

(4, {'polls.Choice': 3, 'polls.Question': 1})

In [109]:
qs = Question.objects.all()
for q in qs:
    print(q.pk, q.question_text, q.pub_date)

2 가고 싶은 나라를 선택하세요. 2021-05-26 17:19:46.670356+00:00
3 좋아하는 동물은 무엇입니까? 2021-05-26 17:23:56.004295+00:00
4  좋아하는 가수는 누구입니까 2021-02-16 05:31:14.059220+00:00


In [110]:
Choice.objects.all()

<QuerySet [<Choice: 영국>, <Choice: 미국>, <Choice: 중국>, <Choice: 강아지>, <Choice: 펭귄>, <Choice: 한국>, <Choice: 브라질>]>

In [122]:
# 특정 조건의 뎅터들을 삭제 => 조회
# choice_text에 색이 들어간 것들 삭제
qs = Choice.objects.filter(choice_text__endswith='색')
for q in qs:
    q.delete()

In [124]:
Choice.objects.all()

<QuerySet [<Choice: 영국>, <Choice: 미국>, <Choice: 중국>, <Choice: 강아지>, <Choice: 펭귄>, <Choice: 한국>, <Choice: 브라질>]>

# 직접 SQL문을 실행
- 모델.objects.raw('select문')
    - RawQuerySet

In [128]:
rq = Choice.objects.raw('select * from polls_choice')
for q in rq:
    print(q, type(q))

영국 <class 'polls.models.Choice'>
미국 <class 'polls.models.Choice'>
중국 <class 'polls.models.Choice'>
강아지 <class 'polls.models.Choice'>
펭귄 <class 'polls.models.Choice'>
한국 <class 'polls.models.Choice'>
브라질 <class 'polls.models.Choice'>
